##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 펭귄 템플릿을 사용하여 데이터에 대한 TFX 파이프라인 생성

---


참고: Google Cloud [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)에서 이 가이드를 실행할 것을 권장합니다. [Vertex AI Workbench로 이동하세요](https://console.cloud.google.com/vertex-ai/workbench).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/penguin_template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

## 소개

이 문서는 TFX Python 패키지와 함께 제공되는 *펭귄 템플릿*을 사용하여 고유 데이터세트에 대한 TensorFlow Extended(TFX) 파이프라인을 만드는 방법을 안내합니다. 생성된 파이프라인은 처음에 [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) 데이터세트를 사용하지만 데이터세트에 대한 파이프라인을 변환합니다.


### 전제 조건

- Linux / MacOS
- Python 3.6-3.8
- Jupyter 노트북


## 1단계. 미리 정의된 템플릿을 프로젝트 디렉터리에 복사합니다.

이 단계에서는 TFX의 *펭귄 템플릿*에서 파일을 복사하여 작업 파이프라인 프로젝트 디렉터리 및 파일을 만듭니다. 이것을 TFX 파이프라인 프로젝트의 스캐폴드로 생각할 수 있습니다.

### Pip 업데이트

Colab에서 실행 중인 경우, 최신 버전의 Pip가 있는지 확인해야 합니다. 물론 로컬 시스템은 별도로 업데이트할 수 있습니다.

참고: Vertex AI Workbench에서 실행 중인 경우 업데이트하는 것도 좋은 생각일 수 있습니다.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### 필수 패키지 설치하기

먼저, TFX와 TensorFlow Model Analysis(TFMA)를 설치합니다.


In [ ]:
!pip install -U tfx tensorflow-model-analysis

TFX 버전을 확인합니다.

In [ ]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tfx

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFX version: {}'.format(tfx.__version__))

파이프라인을 만들 준비가 되었습니다.

`PROJECT_DIR`을 환경에 적합한 대상으로 설정하세요. 기본값은 <a>Google Cloud AI Platform Notebook</a> 환경에 적합한 <code>~/imported/${PIPELINE_NAME}</code>입니다.

아래 `PIPELINE_NAME`을 변경하여 파이프라인에 다른 이름을 지정할 수 있으며, 이는 파일이 저장되는 프로젝트 디렉터리 이름으로도 사용됩니다.


In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Set this project directory to your new tfx pipeline project.
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "imported", PIPELINE_NAME)

### 템플릿 파일을 복사합니다.

TFX에는 TFX python 패키지와 함께 `penguin` 템플릿이 포함되어 있습니다. `penguin` 템플릿에는 이 튜토리얼의 목적인 파이프라인으로 데이터세트를 가져오기 위한 많은 지침이 포함되어 있습니다.

`tfx template copy` CLI 명령은 미리 정의된 템플릿 파일을 프로젝트 디렉토리에 복사합니다.

In [ ]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=penguin

이 노트북의 작업 디렉토리 컨텍스트를 프로젝트 디렉토리로 변경합니다.

In [ ]:
%cd {PROJECT_DIR}

> 참고: JupyterLab 또는 Google Cloud AI Platform Notebook을 사용하는 경우 프로젝트 디렉터리가 생성되면 왼쪽에 있는 `File Browser`에서 디렉터리를 클릭하여 디렉터리를 변경하는 것을 잊지 마세요.

### 복사한 소스 파일 찾아보기

TFX 템플릿은 Python 소스 코드 및 샘플 데이터를 포함하여 파이프라인을 구축하기 위한 기본 스캐폴드 파일을 제공합니다. `penguin` 템플릿은 펭귄 <a>예제</a>와 동일한 <em>Palmer Penguins</em> 데이터세트 및 ML 모델을 사용합니다.

다음은 각 Python 파일에 대한 간략한 소개입니다.

- `pipeline` -이 디렉토리는 파이프라인을 정의합니다.
    - `configs.py` — 파이프라인 러너의 공통 상수를 정의합니다.
    - `pipeline.py` — TFX 컴포넌트 및 파이프라인을 정의합니다.
- `models` - 이 디렉터리에는 ML 모델 정의가 포함되어 있습니다.
    - `features.py`, `features_test.py` — 모델의 특성을 정의합니다.
    - `preprocessing.py`, `preprocessing_test.py` — 데이터의 전처리 루틴을 정의합니다.
    - `constants.py` — 모델 상수를 정의합니다.
    - `model.py`, `model_test.py` — TensorFlow와 같은 ML 프레임워크를 사용하여 ML 모델을 정의합니다.
- `local_runner.py` — 로컬 오케스트레이션 엔진을 사용하는 로컬 환경용 러너를 정의합니다.
- `kubeflow_runner.py` — Kubeflow Pipelines 오케스트레이션 엔진의 러너를 정의합니다.


기본적으로, 템플릿에는 표준 TFX 구성 요소만 포함됩니다. 사용자 지정 작업이 필요한 경우 파이프라인에 대한 사용자 지정 구성 요소를 생성할 수 있습니다. 자세한 내용은 [TFX 사용자 지정 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/understanding_custom_components)를 참조하세요.

#### 단위 테스트 파일.

이름에 `_test.py`가 포함된 일부 파일이 있음을 알 수 있습니다. 이는 파이프라인의 단위 테스트이며 자체 파이프라인을 구현할 때 더 많은 단위 테스트를 추가하는 것이 좋습니다. `-m` 플래그와 함께 테스트 파일의 모듈 이름을 제공하여 단위 테스트를 실행할 수 있습니다. 일반적으로 `.py` 확장자를 삭제하고 `/`를 `.`로 대체하여 모듈 이름을 얻을 수 있습니다. 예를 들면 다음과 같습니다.

In [ ]:
import sys
!{sys.executable} -m models.features_test

### 로컬 환경에서 TFX 파이프라인을 만듭니다.

TFX는 파이프라인을 실행하기 위해 여러 오케스트레이션 엔진을 지원합니다. 우리는 로컬 오케스트레이션 엔진을 사용할 것입니다. 로컬 오케스트레이션 엔진은 더 이상의 종속성 없이 실행되며, 원격 컴퓨팅 클러스터에 의존하지 않고 로컬 환경에서 실행되기 때문에 개발 및 디버깅에 적합합니다.

`local_runner.py`를 사용하여 로컬 오케스트레이터로 파이프라인을 실행합니다. 파이프라인을 먼저 생성해야 실행할 수 있습니다. `pipeline create` 명령을 생성할 수 있습니다.


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

`pipeline create` 명령은 `local_runner.py`에 정의된 파이프라인을 실제로 실행하지 않고 등록합니다.

다음 단계에서 `run create` 명령으로 생성된 파이프라인을 실행합니다.


## 2단계. 자신의 데이터를 파이프라인에 수집합니다.

초기 파이프라인은 템플릿에 포함된 펭귄 데이터세트를 수집합니다. 데이터를 파이프라인에 넣어야 하며 대부분의 TFX 파이프라인은 ExampleGen 구성 요소로 시작합니다.

### ExampleGen 선택

데이터는 로컬 또는 분산 파일 시스템 또는 쿼리 가능한 시스템에서 파이프라인이 액세스할 수 있는 모든 위치에 저장할 수 있습니다. TFX는 데이터를 TFX 파이프라인으로 가져오는 다양한 [`ExampleGen` 구성 요소](https://www.tensorflow.org/tfx/guide/examplegen)를 제공합니다. 구성 요소를 생성하는 다음 예제에서 하나를 선택할 수 있습니다.

- CsvExampleGen: 디렉터리에서 CSV 파일을 읽습니다. [펭귄 예제](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin) 및 [시카고 택시 예제](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)에서 사용됩니다.
- ImportExampleGen: TF 예제 데이터 형식의 TFRecord 파일을 가져옵니다. [MNIST 예제](https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist)에서 사용됩니다.
- [Avro](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/avro_executor.py) 또는 [Parquet](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/parquet_executor.py) 형식을 위한 FileBasedExampleGen
- [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen): Google Cloud BigQuery에서 직접 데이터를 읽습니다. [시카고 택시 예제](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)에서 사용됩니다.

고유한 ExampleGen를 생성할 수도 있습니다. 예를 들어 tfx에는 [Presto를 데이터 소스로 사용하는 사용자 지정 ExecampleGen](https://github.com/tensorflow/tfx/tree/master/tfx/examples/custom_components/presto_example_gen)이 포함되어 있습니다. 사용자 지정 실행기를 사용하고 개발하는 방법에 대한 자세한 내용은 [가이드](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen)를 참조하세요.

사용할 ExampleGen을 결정했으면 데이터를 사용하도록 파이프라인 정의를 수정해야 합니다.

1. `local_runner.py`에서 `DATA_PATH`를 수정하고 파일 위치로 설정합니다.

- 로컬 환경에 파일이 있는 경우 경로를 지정하세요. 이것은 파이프라인을 개발하거나 디버깅하기 위한 최상의 옵션입니다.
- 파일이 GCS에 저장된 경우 `gs://{bucket_name}/...`으로 시작하는 경로를 사용할 수 있습니다. 예를 들어 [`gsutil`](https://cloud.google.com/storage/docs/gsutil)을 사용하여 터미널에서 GCS에 액세스할 수 있는지 확인하세요. 필요한 경우 [Google Cloud의 인증 가이드](https://cloud.google.com/sdk/docs/authorizing)를 따르세요.
- BigQueryExampleGen과 같은 Query 기반 ExampleGen을 사용하려면 데이터 소스에서 데이터를 선택하기 위한 Query 문이 필요합니다. Google Cloud BigQuery를 데이터 소스로 사용하기 위해 설정해야 할 몇 가지 사항이 더 있습니다.
    - `pipeline/configs.py`에서 다음과 같이 합니다.
        - `GOOGLE_CLOUD_PROJECT` 및 `GCS_BUCKET_NAME`을 해당 GCP 프로젝트 및 버킷 이름으로 변경합니다. 파이프라인을 실행하기 전에 버킷이 있어야 합니다.
        - `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` 변수의 주석 처리를 제거합니다.
        - 주석을 제거하고 <strong>쿼리 문</strong>에 <code>BIG_QUERY_QUERY</code> 변수를 설정합니다.
    - `local_runner.py`에서 다음을 수행합니다.
        - `pipeline.create_pipeline()`에서 대신 `data_path` 인수를 주석 처리하고 `query` 인수의 주석 처리를 제거합니다.
    - `pipeline/pipeline.py`에서 다음을 수행합니다.
        - `create_pipeline()`에서 `data_path` 인수를 주석 처리하고 `query` 인수의 주석 처리를 제거합니다.
        - CsvExampleGen 대신 [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen)을 사용합니다.

1. 기존 CsvExampleGen을 `pipeline/pipeline.py`의 ExampleGen 클래스로 교체합니다. 각 ExampleGen 클래스에는 다른 서명이 있습니다. 자세한 내용은 [ExampleGen 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/examplegen)를 참조하세요. `pipeline/pipeline.py`에서 `import` 문으로 필수 모듈을 가져오는 것을 잊지 마세요.

초기 파이프라인은 `ExampleGen`, `StatisticsGen`, `SchemaGen` 및 `ExampleValidator`의 네 가지 요소로 구성됩니다. `StatisticsGen`, `SchemaGen` 및 `ExampleValidator`에 대해 아무것도 변경할 필요가 없습니다. 파이프라인을 처음 실행해 보겠습니다.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

파이프라인이 성공적으로 실행된 경우 "Component ExampleValidator is finished."가 표시되어야 합니다.

### 파이프라인의 출력을 검사합니다.

TFX 파이프라인은 아티팩트와 파이프라인 실행의 메타데이터를 포함하는 [메타데이터 DB(MLMD)](https://www.tensorflow.org/tfx/guide/mlmd)와 아티팩트의 두 가지 출력을 생성합니다. 출력 위치는 `local_runner.py`에 정의되어 있습니다. 기본적으로 아티팩트는 `tfx_pipeline_output` 디렉터리에 저장되고 메타데이터는 `tfx_metadata` 디렉터리 아래에 sqlite 데이터베이스로 저장됩니다.

MLMD API를 사용하여 이러한 출력을 검사할 수 있습니다. 먼저 방금 생성된 출력 아티팩트를 검색하기 위한 몇 가지 유틸리티 함수를 정의합니다.

In [ ]:
import tensorflow as tf
import tfx
from ml_metadata import errors
from ml_metadata.proto import metadata_store_pb2
from tfx.types import artifact_utils

# TODO(b/171447278): Move these functions into TFX library.

def get_latest_executions(store, pipeline_name, component_id = None):
  """Fetch all pipeline runs."""
  if component_id is None:  # Find entire pipeline runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('run')
        if c.properties['pipeline_name'].string_value == pipeline_name
    ]
  else:  # Find specific component runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('component_run')
        if c.properties['pipeline_name'].string_value == pipeline_name and
           c.properties['component_id'].string_value == component_id
    ]
  if not run_contexts:
    return []
  # Pick the latest run context.
  latest_context = max(run_contexts,
                       key=lambda c: c.last_update_time_since_epoch)
  return store.get_executions_by_context(latest_context.id)

def get_latest_artifacts(store, pipeline_name, component_id = None):
  """Fetch all artifacts from latest pipeline execution."""
  executions = get_latest_executions(store, pipeline_name, component_id)

  # Fetch all artifacts produced from the given executions.
  execution_ids = [e.id for e in executions]
  events = store.get_events_by_execution_ids(execution_ids)
  artifact_ids = [
      event.artifact_id for event in events
      if event.type == metadata_store_pb2.Event.OUTPUT
  ]
  return store.get_artifacts_by_id(artifact_ids)

def find_latest_artifacts_by_type(store, artifacts, artifact_type):
  """Get the latest artifacts of a specified type."""
  # Get type information from MLMD
  try:
    artifact_type = store.get_artifact_type(artifact_type)
  except errors.NotFoundError:
    return []
  # Filter artifacts with type.
  filtered_artifacts = [aritfact for aritfact in artifacts
                        if aritfact.type_id == artifact_type.id]
  # Convert MLMD artifact data into TFX Artifact instances.
  return [artifact_utils.deserialize_artifact(artifact_type, artifact)
      for artifact in filtered_artifacts]


from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

import pprint

from tfx.orchestration import metadata
from tfx.types import artifact_utils
from tfx.types import standard_artifacts

def preview_examples(artifacts):
  """Preview a few records from Examples artifacts."""
  pp = pprint.PrettyPrinter()
  for artifact in artifacts:
    print("==== Examples artifact:{}({})".format(artifact.name, artifact.uri))
    for split in artifact_utils.decode_split_names(artifact.split_names):
      print("==== Reading from split:{}".format(split))
      split_uri = artifact_utils.get_split_uri([artifact], split)

      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = [os.path.join(split_uri, name)
                            for name in os.listdir(split_uri)]
      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames,
                                        compression_type="GZIP")
      # Iterate over the first 2 records and decode them.
      for tfrecord in dataset.take(2):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        pp.pprint(example)

import local_runner

metadata_connection_config = metadata.sqlite_metadata_connection_config(
              local_runner.METADATA_PATH)

이제 MLMD에서 출력 아티팩트의 메타데이터를 읽을 수 있습니다.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous pipeline run.
    artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
    # Find artifacts of Examples type.
    examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)
    # Find artifacts generated from StatisticsGen.
    stats_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleStatistics.TYPE_NAME)
    # Find artifacts generated from SchemaGen.
    schema_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Schema.TYPE_NAME)
    # Find artifacts generated from ExampleValidator.
    anomalies_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleAnomalies.TYPE_NAME)

이제 각 구성 요소의 출력을 검사할 수 있습니다. [TFDV(Tensorflow Data Validation)](https://www.tensorflow.org/tfx/data_validation/get_started)는 `StatisticsGen`, `SchemaGen` 및 `ExampleValidator`에서 사용되며 TFDV는 이러한 구성 요소의 출력을 시각화하는 데 사용할 수 있습니다.

이 튜토리얼에서는 TFDV를 내부적으로 사용하여 시각화를 표시하는 TFX의 시각화 헬퍼 메서드를 사용합니다. 각 구성 요소에 대한 자세한 내용은 [TFX 구성 요소 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)을 참조하세요.

#### ExampleGen의 출력 검사하기

ExampleGen의 출력을 살펴보겠습니다. 각 분할에 대한 처음 두 가지 예를 살펴보세요.

In [ ]:
preview_examples(examples_artifacts)

기본적으로 TFX ExampleGen은 예제를 *train* 및 *eval*의 두 분할로 나누지만 [분할 구성을 조정할 수 있습니다](https://www.tensorflow.org/tfx/guide/examplegen#span_version_and_split).

#### StatisticsGen의 출력 검사하기


In [ ]:
visualize_artifacts(stats_artifacts)

이러한 통계는 SchemaGen에 제공되어 데이터 스키마를 자동으로 구성합니다.

#### SchemaGen의 출력 검사하기


In [ ]:
visualize_artifacts(schema_artifacts)

이 스키마는 StatisticsGen의 출력에서 자동으로 추론됩니다. 이 튜토리얼에서는 이 생성된 스키마를 사용하지만 [스키마를 수정하고 사용자 지정할](https://www.tensorflow.org/tfx/guide/statsgen#creating_a_curated_schema) 수도 있습니다.

#### ExampleValidator의 출력 검사하기


In [ ]:
visualize_artifacts(anomalies_artifacts)

이상이 발견되면 모든 예제가 가정을 따른다는 데이터를 검토할 수 있습니다. StatistcsGen과 같은 다른 구성 요소의 출력이 유용할 수 있습니다. 발견된 이상은 파이프라인 실행을 차단하지 않습니다.

`SchemaGen`의 출력에서 사용 가능한 특성을 볼 수 있습니다. `Trainer`에서 직접 ML 모델을 구성하는 데 특성을 사용할 수 있는 경우, 다음 단계를 건너뛰고 4단계로 이동할 수 있습니다. 그렇지 않으면 다음 단계에서 일부 특성 엔지니어링 작업을 수행할 수 있습니다. `Transform` 구성 요소는 평균 계산과 같은 전체 전달 작업이 필요할 때, 특히 확장해야 할 때 필요합니다.

## 3단계. (선택 사항) Transform 구성 요소를 사용한 특성 엔지니어링

이 단계에서는 파이프라인의 `Transform` 구성 요소에서 사용할 다양한 특성 엔지니어링 작업을 정의합니다. 자세한 내용은 [Transform 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/transform)를 참조하세요.

이는 ExampleGen의 출력에서 사용할 수 없는 추가 특성이 훈련 코드에 필요한 경우에만 필요합니다. 그렇지 않으면 Trainer를 사용하는 다음 단계로 빠르게 이동하세요.

### 모델의 특성 정의

`models/features.py`에는 특성 이름, 어휘 크기 등을 포함하여 모델의 특성을 정의하는 상수가 포함되어 있습니다. 기본적으로 `penguin` 템플릿에는 `FEATURE_KEYS` 및 `LABEL_KEY`라는 두 개의 상수가 있습니다. `penguin` 모델은 감독된 훈련을 사용하여 분류 문제를 해결하고 모든 특성이 연속된 숫자 특성이기 때문입니다. 다른 예는 [시카고 택시 예제의 특성 정의](https://github.com/tensorflow/tfx/blob/master/tfx/experimental/templates/taxi/models/features.py)를 참조하세요.


### preprocessing_fn()에서 훈련/서비스 제공을 위한 전처리를 구현합니다.

실제 특성 엔지니어링은 `models/preprocessing.py`의 `preprocessing_fn()` 함수에서 이루어집니다.

`preprocessing_fn`에서 텐서의 입력 dict을 조작하여 텐서의 출력 dict를 생성하는 일련의 함수를 정의할 수 있습니다. TensorFlow Transform API에는 `scale_to_0_1` 및 `compute_and_apply_vocabulary`와 같은 헬퍼 함수가 있거나 단순히 일반 TensorFlow 함수를 사용할 수 있습니다. 기본적으로 `penguin` 템플릿에는 특성 값을 정규화하기 위한 [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score) 함수의 사용 예가 포함되어 있습니다.

<code>preprocessing_fn</code> 작성에 대한 자세한 내용은 <a>Tensflow Transform 가이드</a>를 참조하세요.


### 파이프라인에 Transform 구성 요소를 추가합니다.

preprocessing_fn이 준비되면 파이프라인에 `Transform` 구성 요소를 추가합니다.

1. `pipeline/pipeline.py` 파일에서 `# components.append(transform)` 주석을 제거하여 파이프라인에 구성 요소를 추가합니다.

파이프라인을 업데이트하고 다시 실행할 수 있습니다.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

파이프라인이 성공적으로 실행되면 로그 *어딘가에* "Component Transform is finished."가 표시됩니다. `Transform` 구성 요소와 `ExampleValidator` 구성 요소는 서로 종속되지 않으므로 실행 순서가 고정되어 있지 않습니다. 즉, `Transform` 및 `ExampleValidator` 중 하나가 파이프라인 실행의 마지막 구성 요소가 될 수 있습니다.

### Transform의 출력 검사

Transform 구성 요소는 Tensorflow 그래프와 변환된 예제의 두 가지 출력을 생성합니다. 변환된 예제는 ExampleGen에서도 생성되는 예제 아티팩트 유형이지만 이 예제에는 변환된 특성 값이 대신 포함되어 있습니다.

이전 단계에서 했던 것처럼 이를 검사할 수 있습니다.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous run of Transform component.
    artifacts = get_latest_artifacts(metadata_handler.store,
                                     PIPELINE_NAME, "Transform")
    # Find artifacts of Examples type.
    transformed_examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)

In [ ]:
preview_examples(transformed_examples_artifacts)

## 4단계. Trainer 구성 요소로 모델을 훈련합니다.

`Trainer` 구성 요소를 사용하여 ML 모델을 빌드합니다. 자세한 내용은 [Trainer 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/trainer)를 참조하세요. Trainer 구성 요소에 모델 코드를 제공해야 합니다.

### 모델을 정의합니다.

펭귄 템플릿에서 `models.model.run_fn`은 `Trainer` 구성 요소의 `run_fn` 인수로 사용됩니다. `Trainer` 구성 요소가 실행될 때 `models/model.py`의 `run_fn()` 함수가 호출된다는 의미입니다. 주어진 코드에서 `keras` API를 사용하여 간단한 DNN 모델을 구성하는 코드를 볼 수 있습니다. TFX에서 keras API를 사용하는 방법에 대한 자세한 내용은 [TFX의 TensorFlow 2.x](https://www.tensorflow.org/tfx/guide/keras) 가이드를 참조하세요.

이 `run_fn`에서 모델을 빌드하고 구성 요소에서 지정하는 `fn_args.serving_model_dir`이 가리키는 디렉터리에 이를 저장해야 합니다. `run_fn`에 전달되는 `fn_args`에서 다른 인수를 사용할 수 있습니다. <code>fn_args</code>의 전체 인수 목록은 <a>관련 코드</a>를 참조하세요.

`models/features.py`에서 특성을 정의하고 필요에 따라 사용합니다. 3단계에서 특성을 변환한 경우 변환된 특성을 모델에 대한 입력으로 사용해야 합니다.

### 파이프라인에 Trainer 구성 요소를 추가합니다.

run_fn이 준비되면 `Trainer` 구성 요소를 파이프라인에 추가합니다.

1. `pipeline/pipeline.py` 파일에서 `# components.append(trainer)` 주석을 제거하여 파이프라인에 구성 요소를 추가합니다.

Trainer 구성 요소에 대한 인수는 Transform 구성 요소를 사용하는지 여부에 따라 달라질 수 있습니다.

- <code>Transform</code> 구성 요소를 사용하지 <strong>않는</strong> 경우, 인수를 변경할 필요가 없습니다.

- `Transform` 구성 요소를 사용하는 경우 `Trainer` 구성 요소 인스턴스를 생성할 때 인수를 변경해야 합니다.

    - `examples` 인수를 `examples=transform.outputs['transformed_examples'],`로 변경합니다. 훈련을 위해 변환된 예제를 사용해야 합니다.
    - `transform_graph=transform.outputs['transform_graph'],`와 같은 `transform_graph` 인수를 추가합니다. 이 그래프에는 변환 작업에 대한 TensorFlow 그래프가 포함되어 있습니다.
    - 위와 같이 변경하면 Trainer 구성 요소 생성을 위한 코드는 다음과 같을 것입니다.

    ```python
    # If you use a Transform component.
    trainer = Trainer(
        run_fn=run_fn,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        ...
    ```

파이프라인을 업데이트하고 다시 실행할 수 있습니다.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

이 실행이 성공적으로 처리되면 이제 모델에 대한 첫 번째 TFX 파이프라인을 만들고 실행한 것입니다. 축하합니다!

새 모델은 출력 디렉터리 아래의 특정 위치에 놓여지지만 많은 중간 결과가 보관되어 있는 TFX 파이프라인 외부의 고정 위치 또는 서비스에 모델을 두는 것이 좋습니다. ML 프로덕션 시스템에서 중요한 빌드된 모델을 지속적으로 평가하면 더욱 좋습니다. 다음 단계에서는 TFX에서 지속적인 평가 및 배포가 작동하는 방식을 살펴보겠습니다.

## 5단계. (선택 사항) Evaluator로 모델을 평가하고 pusher로 게시합니다.


[`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) 구성 요소는 `Trainer`에서 빌드된 모든 모델을 지속적으로 평가하고 [`Pusher`](https://www.tensorflow.org/tfx/guide/pusher)는 모델을 파일 시스템의 사전 정의된 위치 또는 [Google Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models)에 복사합니다.

### 파이프라인에 Evaluator 구성 요소를 추가합니다.

`pipeline/pipeline.py` 파일에서 다음을 수행합니다.

1. `# components.append(model_resolver)` 주석을 제거하여 파이프라인에 최신 모델 해석기를 추가합니다. Evaluator는 마지막 파이프라인 실행에서 Evaluator를 전달한 이전 베이스라인 모델과 모델을 비교하는 데 사용할 수 있습니다. `LatestBlessedModelResolver`는 Evaluator를 전달한 최신 모델을 찾습니다.
2. 모델에 적절한 `tfma.MetricsSpec`을 설정합니다. 평가는 ML 모델마다 다를 수 있습니다. 펭귄 템플릿에서는 다중 범주 분류 문제를 해결하는 것이므로 `SparseCategoricalAccuracy`가 사용되었습니다. 또한 특정 슬라이스에 대한 모델을 분석하기 위해 `tfma.SliceSpec`을 지정해야 합니다. 자세한 내용은 [Evaluator 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/evaluator)를 참조하세요.
3. `# components.append(evaluator)` 주석을 제거하여 구성 요소를 파이프라인에 추가합니다.

파이프라인을 업데이트하고 다시 실행할 수 있습니다.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

### Evaluator의 출력 검토하기

이 단계에는 TensorFlow Model Analysis(TFMA) Jupyter 노트북 확장이 필요합니다. TFMA 노트북 확장 버전은 TFMA python 패키지 버전과 동일해야 합니다.

다음 명령은 NPM 레지스트리에서 TFMA 노트북 확장을 설치합니다. 완료하는 데 몇 분 정도 걸릴 수 있습니다.

In [ ]:
# Install TFMA notebook extension.
!jupyter labextension install tensorflow_model_analysis@{tfma.__version__}

설치가 완료되면 **브라우저를 새로 고쳐** 확장 프로그램이 적용되도록 하세요.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
  # Search all aritfacts from the previous pipeline run.
  artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
  model_evaluation_artifacts = find_latest_artifacts_by_type(
      metadata_handler.store, artifacts,
      standard_artifacts.ModelEvaluation.TYPE_NAME)

In [ ]:
if model_evaluation_artifacts:
  tfma_result = tfma.load_eval_result(model_evaluation_artifacts[0].uri)
  tfma.view.render_slicing_metrics(tfma_result)

### 파이프라인에 Pusher 구성 요소를 추가합니다.

모델이 유망해 보이면 모델을 게시해야 합니다. [Pusher 구성 요소](https://www.tensorflow.org/tfx/guide/pusher)는 [사용자 지정 실행기](https://github.com/tensorflow/tfx/blob/master/tfx/extensions/google_cloud_ai_platform/pusher/executor.py)를 사용하여 파일 시스템의 위치 또는 GCP AI Platform Models에 모델을 게시할 수 있습니다.

<a><code data-md-type="codespan">Evaluator</code></a> 구성 요소는 <code>Trainer</code>에서 빌드된 모든 모델을 지속적으로 평가하고 <a><code>Pusher</code></a>는 모델을 파일 시스템의 사전 정의된 위치 또는 <a>Google Cloud AI Platform Models</a>에 복사합니다.

1. `local_runner.py`에서 `SERVING_MODEL_DIR`을 게시할 디렉터리로 설정합니다.
2. `pipeline/pipeline.py` 파일에서 `# components.append(pusher)` 주석을 제거하여 파이프라인에 Pusher를 추가합니다.

파이프라인을 업데이트하고 다시 실행할 수 있습니다.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

`SERVING_MODEL_DIR`에서 새 모델을 찾을 수 있습니다.

## 6단계. (선택 사항) 파이프라인을 GCP의 Kubeflow Pipelines에 배포합니다.


앞서 언급했듯이 `local_runner.py`는 디버깅 또는 개발 목적으로 적합하지만 프로덕션 워크로드에 최상의 솔루션은 아닙니다. 이 단계에서는 파이프라인을 Google Cloud의 Kubeflow Pipelines에 배포합니다.

### 준비

Kubeflow Pipelines 클러스터에 파이프라인을 배포하려면 `kfp` python 패키지와 `skaffold` 프로그램이 필요합니다.

In [ ]:
!pip install --upgrade -q kfp

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold

쉘이 찾을 수 있는 위치로 `skaffold` 바이너리를 옮겨야 합니다. 또는 `--skaffold-cmd` 플래그로 `tfx` 바이너리를 실행할 때 skaffold의 경로를 지정할 수 있습니다.

In [ ]:
# Move skaffold binary into your path
!mv skaffold /home/jupyter/.local/bin/

파이프라인을 실행하려면 Kubeflow Pipelines 클러스터도 필요합니다. [Cloud AI Platform 파이프라인의 TFX](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines) 가이드에 나와 있는 1단계와 2단계를 따르세요.

클러스터가 준비되면 <a>Google 클라우드 콘솔의 <code>Pipelines</code></a> 페이지에서 <em>파이프라인 대시보드 열기</em>를 클릭하여 파이프라인 대시보드를 엽니다. 이 페이지의 URL은 파이프라인 실행을 요청하는 `ENDPOINT`입니다. 엔드포인트 값은 URL에서 https:// 이후의 googleusercontent.com까지 모든 값입니다. 엔드포인트를 다음 코드 블록에 넣습니다.


In [ ]:
ENDPOINT='' # Enter your ENDPOINT here.

Kubeflow Pipelines 클러스터에서 코드를 실행하려면 코드를 컨테이너 이미지로 압축해야 합니다. 이미지는 파이프라인을 배포하는 동안 자동으로 빌드되며 이미지의 이름과 컨테이너 레지스트리만 설정하면 됩니다. 이 예에서는 [Google Container Registry](https://cloud.google.com/container-registry) 를 사용하고 이름을 `tfx-pipeline`으로 지정합니다.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]

# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

### 데이터 위치를 설정합니다.

Kubeflow Pipelines 클러스터에서 데이터에 액세스할 수 있어야 합니다. 로컬 환경에서 데이터를 사용한 경우 Google Cloud Storage와 같은 원격 저장소에 데이터를 업로드해야 할 수 있습니다. 예를 들어 다음과 같이 Kubeflow Pipelines 클러스터가 배포될 때 자동으로 생성되는 기본 버킷에 펭귄 데이터를 업로드할 수 있습니다.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/penguin/

`kubeflow_runner.py`의 `DATA_PATH`에 저장된 데이터 위치를 업데이트합니다.

BigQueryExampleGen을 사용하는 경우 데이터 파일을 업로드할 필요는 없지만 `beam_pipeline_args`가 `pipeline.create_pipeline()` 함수에 대해 동일한 `query` 및 `kubeflow_runner.py` 인수를 사용하도록 해야 합니다.

### 파이프라인을 배포합니다.

모든 것이 준비되면 `tfx pipeline create` 명령을 사용하여 파이프라인을 생성할 수 있습니다.

> 참고: Kubeflow Pipelines용 파이프라인을 생성할 때 파이프라인을 실행하는 데 사용할 컨테이너 이미지가 필요합니다. 그리고 `skaffold`는 이미지를 빌드해줍니다. `skaffold`는 도커 허브에서 기본 이미지를 가져오기 때문에 이미지를 처음 빌드할 때는 5~10분 정도 걸리지만 두 번째 빌드에서는 시간이 훨씬 적게 걸립니다.


In [ ]:
!tfx pipeline create  \
--engine=kubeflow \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

이제 `tfx run create` 명령을 사용하여 새로 생성된 파이프라인으로 실행을 시작합니다.

In [ ]:
!tfx run create --engine=kubeflow --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

또는 Kubeflow Pipelines 대시보드에서 파이프라인을 실행할 수도 있습니다. 새 실행은 Kubeflow Pipelines 대시보드의 `Experiments` 아래에 나열됩니다. 실험을 클릭하면 진행 상황을 모니터링하고 실행이 진행되는 동안 생성된 아티팩트를 시각화할 수 있습니다.

Kubeflow Pipelines에서 파이프라인을 실행하는 데 관심이 있다면 [Cloud AI Platform Pipelines의 TFX 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines)에서 자세한 지침을 찾아보세요.

### 정리

이 단계에서 사용된 모든 Google Cloud 리소스를 정리하기 위해 튜토리얼에 사용한 [Google Cloud 프로젝트를 삭제할](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) 수 있습니다.

또는 아래의 각 콘솔로 이동하여 개별 리소스를 정리할 수 있습니다.

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)